In [ ]:
import numpy as np
import pandas as pd

def time_plot_una_estacion(codigo_estacion, columna, fecha_min, fecha_max):

    """
    Grafica datos de 1 columna para 1 estación en intervalo de fechas definidas 
    por fecha_min y fecha_max

    Parametros
    ----------
    codigo_estacion : int
    columna: str, 'caudal', 'precip_promedio' o 'temp_max_promedio'
    fecha_min, fecha_max: str de fechas en formato 'YYYY-MM-dd'

    -------
    """    
    
    fecha_min = np.datetime64(str(fecha_min), 'h')
    fecha_max = np.datetime64(str(fecha_max), 'h')

    datos_estacion = pd.read_csv('caudal_extra.csv', header=0)
    datos_estacion = datos_estacion.sort_values(by=['nombre', 'fecha'])
    datos_estacion = datos_estacion[datos_estacion['codigo_estacion']==int(codigo_estacion)]
    
    #Seleccion de fecha
    datos_estacion = datos_estacion[datos_estacion['fecha'].astype('datetime64[h]')>=fecha_min]
    datos_estacion = datos_estacion[datos_estacion['fecha'].astype('datetime64[h]')<=fecha_max]
    
    x = datos_estacion.fecha.values.astype('datetime64[D]')
    y = datos_estacion[str(columna)]
        
    plt.figure()
    plt.plot(x,y)
    plt.title('Registros estación '+str(codigo_estacion))
    plt.ylabel(str(columna))
    plt.xlabel('Fecha')
    
    return None

#time_plot_una_estacion(11335002, 'caudal', '2017-01', '2017-08')


def time_plot_estaciones_varias_columnas(codigo_estacion, columnas, fecha_min, fecha_max):
    
    """
    Grafica datos de columnas para 1 estación en intervalo de fechas definidas 
    por fecha_min y fecha_max, normalizados por el valor máximo de la serie

    Parametros
    ----------
    codigo_estacion : int
    columnas: lista de str, puede tener: 'caudal', 'precip_promedio' o 'temp_max_promedio'
    fecha_min, fecha_max: str de fechas en formato 'YYYY-MM-dd'

    -------
    """    
    
    fecha_min = np.datetime64(str(fecha_min), 'h')
    fecha_max = np.datetime64(str(fecha_max), 'h')

    datos_estacion = pd.read_csv('caudal_extra.csv', header=0)
    datos_estacion = datos_estacion.sort_values(by=['nombre', 'fecha'])
    datos_estacion = datos_estacion[datos_estacion['codigo_estacion']==int(codigo_estacion)]
    
    #Seleccion de fecha
    datos_estacion = datos_estacion[datos_estacion['fecha'].astype('datetime64[h]')>=fecha_min]
    datos_estacion = datos_estacion[datos_estacion['fecha'].astype('datetime64[h]')<=fecha_max]
    
    x = datos_estacion.fecha.values.astype('datetime64[D]')
        
    plt.figure()
    for i in columnas:
        y = datos_estacion[str(i)]/np.nanmax(datos_estacion[str(i)])
        plt.plot(x,y)
    plt.title('Registros estación '+str(codigo_estacion))
    plt.legend(columnas)
    plt.xlabel('Fecha')
    
    return None

#time_plot_estaciones_varias_columnas(11335002, ['caudal', 'precip_promedio', 'temp_max_promedio'], '2017-01', '2017-08')

def evento_extremo(x, interval_c = 0.95):
     
    """
    Determina valores en array x que estén fuera del intervalo de confianza (interval_c) 
    considerando distribución normal

    Asigna valor de 0 a elementos dentro del intervalo    
    Asigna valor de 1 a elementos fuera del intervalo

    Parametros
    ----------
    x : array 1-d
    interval_c: float entre 0 y 1

    -------
    Retorna
    
    array con labels para cada elemento del array x
    """    
    nan_index = np.isnan(x)
    x_sin_nan = x[~nan_index]    
    
    restos =  (1-interval_c)/2
    lim_menor = norm.ppf(restos)    
    lim_mayor = norm.ppf(interval_c+restos)
    x_norm = (x_sin_nan - np.mean(x_sin_nan))/np.std(x_sin_nan)
    
    labels = np.zeros(len(x))
    labels[~nan_index] = -1*(np.logical_and(x_norm >= lim_menor, x_norm <= lim_mayor).astype(int)-1)
    labels[nan_index] = np.nan
    
    return labels